# se detectan vehiculos sin reducir calidad de video y escogiendo un framerate, se almacenan las imagenes en "detected_vehicles_images"


In [ ]:
import cv2
import os
from ultralytics import YOLO
from datetime import datetime

# Cargar el modelo YOLO v11 (usando "yolo11n.pt")
model = YOLO("yolo11n.pt")

# Definir las clases de vehículos (según el mapeo COCO: car=2, motorcycle=3, bus=5, truck=7)
vehicle_class_indices = [2, 3, 5, 7]

# Crear carpeta para guardar las imágenes de vehículos detectados
output_folder = "detected_vehicles_images"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Definir la ruta del video a procesar y abrirlo
video_path = 'videos/calle1.mp4'
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Error al abrir el video.")
    exit()

# Obtener propiedades iniciales del video
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps    = cap.get(cv2.CAP_PROP_FPS)
print("Video abierto. FPS =", fps, "| Tamaño:", height, "x", width)

frame_count = 0
vehicle_image_count = 0

# Definir el intervalo de frames en el que se usará el modelo (por ejemplo, cada 10 frames)
frame_interval = 10

while True:
    ret, frame = cap.read()
    if not ret:
        print("Fin del video.")
        break

    # Corregir la orientación del frame aplicando una rotación
    frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
    
    frame_count += 1

    # Procesar solo el frame si se cumple el intervalo deseado
    if frame_count % frame_interval != 0:
        continue

    # Usar el frame original sin reducción de calidad para realizar la detección
    results = model(frame, imgsz=640, stream=False, verbose=False, conf=0.5, classes=vehicle_class_indices)
    
    if results and results[0].boxes is not None:
        # Obtener las detecciones: [x1, y1, x2, y2, confidence, class]
        dets = results[0].boxes.data.cpu().numpy() if hasattr(results[0].boxes.data, "cpu") else results[0].boxes.data
        for det in dets:
            x1, y1, x2, y2, conf, cls = det
            cls = int(cls)
            if cls in vehicle_class_indices:
                # Asegurarse de que las coordenadas sean enteras y estén dentro de la imagen
                x1_orig = max(int(x1), 0)
                y1_orig = max(int(y1), 0)
                x2_orig = min(int(x2), frame.shape[1])
                y2_orig = min(int(y2), frame.shape[0])
                
                # Extraer la región de interés (ROI) del vehículo
                roi = frame[y1_orig:y2_orig, x1_orig:x2_orig]
                
                # Generar un nombre único para la imagen usando la fecha y un contador
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
                filename = os.path.join(output_folder, f"vehicle_{timestamp}_{vehicle_image_count}.png")
                cv2.imwrite(filename, roi)
                vehicle_image_count += 1

                # (Opcional) Dibujar un rectángulo en el frame original para visualizar la detección
                cv2.rectangle(frame, (x1_orig, y1_orig), (x2_orig, y2_orig), (0, 255, 0), 4)

cap.release()


Video abierto. FPS = 29.99066484427621 | Tamaño: 1080 x 1920
Fin del video.


## implementacion con deepsort_realtime

In [7]:
import cv2
import os
import numpy as np
from ultralytics import YOLO
from datetime import datetime
from deep_sort_realtime.deepsort_tracker import DeepSort

# Cargar el modelo YOLO (en este ejemplo "yolo11n.pt")
model = YOLO("yolo11n.pt")
# Clases de vehículos (COCO: car=2, motorcycle=3, bus=5, truck=7)
vehicle_class_indices = [2, 3, 5, 7]

# Inicializar DeepSORT
tracker = DeepSort(max_age=30)

# Configurar video de entrada
video_path = 'videos/calle4.mp4'
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Error al abrir el video.")
    exit()

# Propiedades del video
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps    = cap.get(cv2.CAP_PROP_FPS)
print("Video abierto. FPS =", fps, "| Tamaño:", height, "x", width)

# Configurar video de salida (recordar que se rota el frame, por lo que ancho y alto pueden intercambiarse)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output_tracked.mp4', fourcc, fps, (height, width))

frame_count = 0
frame_interval = 1

def get_color(idx):
    """Genera un color único para cada ID (nota: se reinicia la semilla para consistencia)."""
    np.random.seed(idx)
    return tuple(np.random.randint(0, 255, 3).tolist())


def get_color(idx):
    """Genera un color único para cada ID."""
    # Si idx es numérico en forma de string, conviértelo a entero.
    try:
        seed = int(idx)
    except ValueError:
        # Si no se puede convertir directamente, se puede usar hash y reducirlo.
        seed = hash(idx) % (2**32)
    np.random.seed(seed)
    return tuple(np.random.randint(0, 255, 3).tolist())

while True:
    ret, frame = cap.read()
    if not ret:
        print("Fin del video.")
        break

    # Rotar el frame para corregir la orientación
    frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
    frame_count += 1

    # Procesar el frame según un intervalo (por ejemplo, cada 10 frames)
    if frame_count % frame_interval != 0:
        out.write(frame)
        continue

    # Realizar detección con YOLO en las clases de vehículos
    results = model(frame, imgsz=640, stream=False, verbose=False, conf=0.5, classes=vehicle_class_indices)

    detections_for_tracker = []
    if results and results[0].boxes is not None:
        # Extraer detecciones: [x1, y1, x2, y2, conf, cls]
        dets = results[0].boxes.data.cpu().numpy() if hasattr(results[0].boxes.data, "cpu") else results[0].boxes.data
        for det in dets:
            x1, y1, x2, y2, conf, cls = det
            # Convertir la caja a formato [x, y, ancho, alto]
            x1 = max(int(x1), 0)
            y1 = max(int(y1), 0)
            w = int(x2) - int(x1)
            h = int(y2) - int(y1)
            bbox = [x1, y1, w, h]
            # Se asigna una etiqueta genérica "vehicle"
            detections_for_tracker.append((bbox, conf, "vehicle"))

    # Actualizar el tracker con las detecciones actuales y el frame (opcional para el modelo de features)
    tracks = tracker.update_tracks(detections_for_tracker, frame=frame)

    # Dibujar bounding boxes y asignar un color único por ID
    for track in tracks:
        # Solo consideramos tracks confirmados
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        l, t, r, b = track.to_ltrb()  # Coordenadas: left, top, right, bottom
        color = get_color(track_id)
        cv2.rectangle(frame, (int(l), int(t)), (int(r), int(b)), color, 4)
        cv2.putText(frame, f'ID: {track_id}', (int(l), int(t)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Guardar el frame procesado en el video de salida
    out.write(frame)

cap.release()
out.release()


Video abierto. FPS = 23.976023976023978 | Tamaño: 720 x 1280
Fin del video.


In [ ]:
import os
import 

print(os.path.isdir(labels_dir))

NameError: name 'labels_dir' is not defined